# Read data from NVDB and convert to RDF for comparision with Overture Maps

Libraries and parameters

In [1]:
import sys, datetime
from rdflib import Graph, Namespace
import nvdbapiv3
from nvdb2rdf import *
import geopandas as gpd
from shapely import Point, wkt, LineString
from shapely.geometry import shape, box
from shapely.wkb import loads
import pandas as pd
import pyproj
import overturemaps
from lonboard import Map, PolygonLayer, PathLayer, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
from lonboard import viz
import geopandas as gpd

Føyer C:\DATA\GitHub\vegvesen\NVDB-Datakatalogen\owl til søkestien


In [3]:
# Parameters
rootPath = "C:\\Data\\GitHub\\jetgeo\\OM2ANY"

#NVDB Paths
nvdbOtlFilePath = "C:\\Data\\GitHub\\jetgeo\\OM2ANY\\OWL\\nvdb\\nvdb-owl.ttl"
nvdbVoIRI = "https://ontologi.atlas.vegvesen.no/nvdb/core/nvdb-owl/vegobjekt#"
nvdbVnIRI = "https://ontologi.atlas.vegvesen.no/nvdb/core/vegnett#"
nvdbOtlIRI = "https://ontologi.atlas.vegvesen.no/nvdb/core/nvdb-owl#"

#INSPIRE Paths
itnrOtlFilePath = "C:\\Data\\GitHub\\jetgeo\\OM2ANY\\OWL\\inspire\\itnr-owl.ttl"
itnrOtlIRI = "http://inspire.ec.europa.eu/ont"
featuretypeid = 105
knr = 3403
utsnitt = 'modi.geojson'

Read ontologies

In [ ]:
# ---------------------------------------------------------------------------------------------
startTime = datetime.datetime.now()
# ---------------------------------------------------------------------------------------------
# Leser NVDB-ontologien til graf for objekttypebiblioteker
print(str(datetime.datetime.now()) + ' Leser inn NVDB-OTL fra ', nvdbOtlFilePath)
otl_nvdb = Graph()
otl_nvdb.parse(nvdbOtlFilePath, format="turtle")
# Leser INSPIRE-ontologien til egen graf
print(str(datetime.datetime.now()) + ' Leser inn INSPIRE-OTL fra ', itnrOtlFilePath)
otl_itnr = Graph()
otl_itnr.parse(itnrOtlFilePath, format="turtle")
#Slår sammen alle ontologier til en stor graf
print(str(datetime.datetime.now()) + ' Slår sammen ontologiene')
otl = Graph()
otl = otl_nvdb + otl_itnr 
# Setter opp graf og namespace-forkortelser for NVDB-data
print(str(datetime.datetime.now()) + ' Setter opp graf og namespace-forkortelser for data')
g_nvdb=Graph()
g_nvdb.bind("nvdb_vo", Namespace(nvdbVoIRI)) #IRI for NVDB-Objekter
g_nvdb.bind("nvdb_vn", Namespace(nvdbVnIRI)) #IRI for NVDB Vegnett
g_nvdb.bind("nvdb_otl",Namespace(nvdbOtlIRI))
g_nvdb.bind("gsp",'http://www.opengis.net/ont/geosparql#')
g_nvdb.bind("nvdb_otl",Namespace(nvdbOtlIRI))


# ---------------------------------------------------------------------------------------------

Read data from NVDB and convert to RDF

In [ ]:
g_nvdb = nvdb2graph(featuretypeid, knr, otl_nvdb)

Print to Turtle

In [ ]:
fileName = rootPath + "\\data\\nvdb_" + str(featuretypeid) + "_" + str(knr) + ".ttl."
print(str(datetime.datetime.now()) + ' Skriver til NVDB-Turtle-fil: ' + fileName)
g_nvdb.serialize(destination=fileName, format="turtle")

Read road network from NVDB and create GeoDataFrames with links and nodes

In [4]:
# Read GeoJSON Polygon, transform to EPSG:5973 and use for filtering NVDB
gjFile = rootPath + "\\Data\\" + utsnitt

# Bounding box
import geojson
from shapely.geometry import shape

def get_bbox(geometry):
    polygon = shape(geometry)
    return polygon.bounds

gjP4326 = gpd.read_file(gjFile)
gjP5973 = gjP4326.to_crs(epsg=5973)

polygon_coords = gjP5973.geometry.iloc[0].exterior.coords

# Create a string with the coordinates separated by commas
coords_string = ", ".join([f"{x:.6f} {y:.6f}" for x, y in polygon_coords])

# Print the coordinates string
print(f"Coordinates of the polygon feature: {coords_string}")

Coordinates of the polygon feature: 283646.630245 6550974.166444, 290696.263405 6552761.073827, 290798.107492 6559552.695475, 286687.386750 6571174.505270, 278959.847715 6582843.761427, 261459.965387 6592816.360618, 261684.402907 6602897.758014, 253986.688583 6604474.483078, 252409.898995 6592726.902924, 262426.088473 6584522.198264, 274228.569390 6573790.349457, 280837.881236 6561058.296764, 283646.630245 6550974.166444


In [5]:
# Read from NVDB
v = nvdbapiv3.nvdbVegnett()
v.filter({'polygon' : coords_string})
# v.filter( { 'kommune' : knr } )
print("Filter: " + str(v.filterdata))

#Convert to GeoDataFrame
sDf = pd.DataFrame(v.to_records())
#Remove 'vegtrase' links
sDf = sDf[sDf['detaljnivå'] != 'Vegtrase']
sDf['geometry'] = sDf['geometri'].apply( wkt.loads )
sGDF = gpd.GeoDataFrame( sDf, geometry='geometry', crs=5973 )

# Convert start and end coordinates to Shapely Point geometries
startpoints = [Point(coords[0]) for coords in sGDF['geometry'].apply(lambda line: line.coords)]
endpoints = [Point(coords[-1]) for coords in sGDF['geometry'].apply(lambda line: line.coords)]
# Create GeoDataFrame with nodes 
nGDF = gpd.GeoDataFrame({
    'geometry': startpoints + endpoints,
    'nodeid': sGDF['startnode'].tolist() + sGDF['sluttnode'].tolist(),
}, crs=sGDF.crs)
# Remove duplicate nodes in the GeoDataFrame
nGDF.drop_duplicates(subset='nodeid', inplace=True)

#Write to GeoJSON files
sGDF.to_file(rootPath + "\\data\\nvdb\\nvdb_Segments.geojson", driver="GeoJSON")
nGDF.to_file(rootPath + "\\data\\nvdb\\nvdb_Nodes.geojson", driver="GeoJSON")

Filter: {'polygon': '283646.630245 6550974.166444, 290696.263405 6552761.073827, 290798.107492 6559552.695475, 286687.386750 6571174.505270, 278959.847715 6582843.761427, 261459.965387 6592816.360618, 261684.402907 6602897.758014, 253986.688583 6604474.483078, 252409.898995 6592726.902924, 262426.088473 6584522.198264, 274228.569390 6573790.349457, 280837.881236 6561058.296764, 283646.630245 6550974.166444'}


Find nodes that connects more than two road segments

In [6]:
# Create a DataFrame to count the occurrences of each node in the 'startnode' and 'endnode' columns
node_counts = pd.concat([
    sGDF['startnode'].value_counts(),
    sGDF['sluttnode'].value_counts()
], axis=1, keys=['startnode', 'sluttnode']).fillna(0)

# Filter nodes that connect to more than 2 segments
multi_segment_nodes = node_counts[(node_counts['startnode'] + node_counts['sluttnode']) > 2].index.tolist()
# Create the new node GeoDataFrame based on 'nodeid'
filtered_nodes_gdf = nGDF[nGDF['nodeid'].isin(multi_segment_nodes)]
filtered_nodes_gdf.to_file(rootPath + "\\data\\nvdb\\nvdb_NodesFiltered.geojson", driver="GeoJSON")
print(filtered_nodes_gdf)

                                      geometry   nodeid
0      POINT Z (256058.115 6604216.822 37.710)   458246
1      POINT Z (256114.690 6604114.350 37.030)  2039960
4      POINT Z (257061.129 6603021.996 22.954)   458897
5      POINT Z (257150.412 6602981.498 24.162)  2065319
6      POINT Z (257187.430 6602963.900 21.020)   458973
...                                        ...      ...
70218  POINT Z (278575.490 6577371.360 37.470)  3509366
70264  POINT Z (278049.620 6576938.980 27.410)  3509377
70336  POINT Z (278865.050 6578302.710 40.470)  3514659
71206   POINT Z (254352.200 6596969.150 3.770)  3607667
72008  POINT Z (286761.730 6560473.970 28.910)  3819706

[15323 rows x 2 columns]


Convert to NVDB data to WGS 84 (EPSG:4326)

In [7]:
sGDF4326 = sGDF.to_crs(epsg=4326)
sGDF4326.crs="EPSG:4326"
nGDF4326 = nGDF.to_crs(epsg=4326)
nGDF4326.crs="EPSG:4326"
#nGDF4326.to_file(rootPath + "\\data\\nvdb\\nvdb_Nodes4326.geojson", driver="GeoJSON")

Find extent of the NVDB Data and create a bounding box in EPSG:4326


In [8]:
# Get the overall bounding box (extent) for all geometries
minx, miny, maxx, maxy = sGDF.total_bounds

# Create a Shapely bounding box
bounding_box = box(minx, miny, maxx, maxy)

# Create a PyProj transformer for EPSG:5973 to EPSG:4326
transformer = pyproj.Transformer.from_crs("EPSG:5973", "EPSG:4326", always_xy=True)

# Transform the bounding box coordinates to EPSG:4326
minlon, minlat = transformer.transform(minx, miny)
maxlon, maxlat = transformer.transform(maxx, maxy)

# Create a  bounding box in EPSG:4326
bbox = minlon, minlat, maxlon, maxlat
print(bbox)

(10.607018549727396, 59.07095720324578, 11.310705049506575, 59.531880931884736)


Read Overture Segments and Nodes within the BBox and store in GeoDataFrame

In [9]:
def rbr2GDF(rbr):
    #Convert Record Batch Reader to GeoDataFrame
    # Extract the binary geometry column
    binary_geometry = rbr['geometry']
    # Convert binary geometry to Shapely geometries
    geometries = [loads(geom.as_py()) for geom in binary_geometry]
    # Create a Pandas DataFrame with the geometries 
    df = rbr.to_pandas()
    df['geometry'] = geometries
    # df['id'] = df['id']  
    # Create a GeoDataFrame with the geometries 
    gdf = gpd.GeoDataFrame(df, geometry='geometry',crs=4326)
    return gdf

ft= "segment"
segTable = overturemaps.record_batch_reader(ft, bbox).read_all()
# Temporarily required as of Lonboard 0.8 to avoid a Lonboard bug
segTable = segTable.combine_chunks()
segGDF = rbr2GDF(segTable)
# print(segGDF.head(10))

ft= "connector"
conTable = overturemaps.record_batch_reader(ft, bbox).read_all()
conGDF = rbr2GDF(conTable)
# print(conGDF.head(10))

In [10]:
# Clip within polygon
segGDF_clipped = gpd.sjoin(segGDF, gjP4326, predicate='within')
segGDF = segGDF_clipped
conGDF_clipped = gpd.sjoin(conGDF, gjP4326, predicate='within')
conGDF = conGDF_clipped


Export to GeoJSON, for use in QGIS

In [11]:
gdf_filtered = segGDF[['id', 'geometry']]
gdf_filtered.to_file(rootPath + "\\data\\om\\om_segments.geojson", driver="GeoJSON")
gdf_filtered = conGDF[['id', 'geometry']]
gdf_filtered.to_file(rootPath + "\\data\\om\\om_connectors.geojson", driver="GeoJSON")

Show in lonboard

In [12]:
segLayer = PathLayer(
    table=segTable, get_color=[0, 0, 139], width_min_pixels=1
)
conLayer = ScatterplotLayer(
    table=conTable,get_fill_color=[255, 0, 0]
)

nodeLayer = ScatterplotLayer.from_geopandas(conGDF,get_fill_color=[155, 75, 0]
)
sLayer = PathLayer.from_geopandas(segGDF, get_color=[0, 0, 139], width_min_pixels=1)

view_state = {
    "longitude": 11.08,
    "latitude": 60.795,
    "zoom": 13,
    "pitch": 0,
    "bearing": 0,
}

m = Map([sLayer], view_state=view_state,_height=1000)
# m = Map([segLayer,conLayer], view_state=view_state,_height=1000)
m

#Note: lonboard fucks up the presentation of NVDB data, wrong positioning

C:\Users\JETKNU\AppData\Roaming\Python\Python312\site-packages\lonboard\_geoarrow\ops\reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(layers=[PathLayer(get_color=[0, 0, 139], table=pyarrow.Table
id: string
bbox: struct<xmax: double, xmin: d…

Compare Nodes

In [19]:
conOM5973 = conGDF.to_crs(epsg=5973)
conOM5973.crs="EPSG:5973"

# Find points in gdf1 that are not within max_deviation distance of any point in gdf2:
# Reset the indices of both GeoDataFrames
filtered_nodes_gdf = filtered_nodes_gdf.reset_index(drop=True)
conOM5973 = conOM5973.reset_index(drop=True)

# Buffer the `conOM5973` GeoDataFrame by 3.0 meter
buffered_conOM5973 = conOM5973.buffer(3.0)
# Assuming `buffered_conOM5973` is a GeoSeries, convert it to a GeoDataFrame
buffered_conOM5973_gdf = gpd.GeoDataFrame(geometry=buffered_conOM5973)

# Perform a spatial join
joined_df = gpd.sjoin(filtered_nodes_gdf, buffered_conOM5973_gdf, how="left", predicate="within")

# Filter out points that did not intersect with any point in `conOM5973`
missing_in_om = joined_df[joined_df["index_right"].isna()]

# Save the result to a GeoJSON file
missing_in_om.to_file(rootPath + "\\data\\nvdb\\missing_in_om.geojson", driver="GeoJSON")

print(missing_in_om)

                                      geometry   nodeid  index_right
0      POINT Z (256058.115 6604216.822 37.710)   458246          NaN
1      POINT Z (256114.690 6604114.350 37.030)  2039960          NaN
3      POINT Z (257150.412 6602981.498 24.162)  2065319          NaN
8      POINT Z (257828.160 6602009.950 58.955)   459406          NaN
11     POINT Z (256427.400 6604624.200 51.809)   458569          NaN
...                                        ...      ...          ...
15314  POINT Z (278839.180 6580160.910 56.360)  3506961          NaN
15315  POINT Z (276744.950 6579042.900 35.200)  3507634          NaN
15316  POINT Z (276899.550 6578927.560 30.290)  3507657          NaN
15318  POINT Z (278575.490 6577371.360 37.470)  3509366          NaN
15322  POINT Z (286761.730 6560473.970 28.910)  3819706          NaN

[5111 rows x 3 columns]


In [ ]:
#TODO: Compare restrictions

# 1. Derive geometry for speedlimits, access and prohibited transitions

# Linear referencing in Python: 
# - ShapelyM? https://pypi.org/project/shapelyM/
# - linref? https://pypi.org/project/linref/
# - https://shapely.readthedocs.io/en/stable/manual.html#linear-referencing-methods

# 2. Export to GeoJSON for viewing in QGIS
# 3. Find NVDB-LR for the geometries

    # line_df['geometry'] = line_df.apply(lambda row: LineString([(row[start_m], 0), (row[end_m], 0)]), axis=1)
from shapely.geometry import LineString

# Example LineString (replace with your actual data)
road_segment = LineString([(0, 0), (10, 0)])

# Calculate the split distances (10% from both ends)
split_distance_start = 0.1 * road_segment.length
split_distance_end = 0.9 * road_segment.length

# Create two new LineStrings
part1 = LineString(road_segment.coords[:int(split_distance_start)])
part2 = LineString(road_segment.coords[int(split_distance_end):])

print(part1)  # The remaining part after removing the first 10%
print(part2)  # The remaining middle part